In [1]:
import numpy as np
import trimesh
import tqdm 

In [2]:
def uniform_sampling_from_mesh(vertices, faces, sample_num):
    # -------- TODO -----------
    # 1. compute area of each triangles
    # 2. compute probability of each triangles from areas
    # 3. sample N faces according to the probability
    # 4. for each face, sample 1 point
    # Note that FOR-LOOP is not allowed!
    v_0, v_1, v_2 = vertices[faces[:, 0], :], vertices[faces[:, 1], :], vertices[faces[:, 2], :]
    area = np.linalg.norm(np.cross(v_1 - v_0, v_2 - v_0), axis = 1) / 2
    prob = area / np.sum(area)
    idx_sampled_faces = np.random.choice(faces.shape[0], size = sample_num, p = prob)
    r_1 = np.random.uniform(0, 1, (sample_num, 1))
    r_2 = np.random.uniform(0, 1, (sample_num, 1))
    uniform_pc = (1 - r_1 ** 0.5) * v_0[idx_sampled_faces] + \
        r_1 ** 0.5 * (1 - r_2) * v_1[idx_sampled_faces] + r_1 ** 0.5 * r_2 * v_2[idx_sampled_faces]
    # -------- TODO -----------
    return area, prob, uniform_pc
        

In [3]:
def farthest_point_sampling(pc, sample_num):
    # -------- TODO -----------
    # FOR LOOP is allowed here.
    idx_0 = np.random.choice(pc.shape[0], size = 1)[0]
    p_0 = pc[idx_0]
    results = p_0[np.newaxis, :]
    distance_S = np.linalg.norm(p_0 - pc, axis = 1)
    for _ in  range(sample_num - 1):
        p_new = pc[np.argmax(distance_S)]
        results = np.concatenate([results, p_new[np.newaxis, :]], axis = 0)
        distance_p_new = np.linalg.norm(p_new - pc, axis = 1)
        distance_S = np.where(distance_S > distance_p_new, distance_S, distance_p_new)
    # -------- TODO -----------

    return results

In [4]:
# task 1: uniform sampling 

obj_path = 'spot.obj'
mesh = trimesh.load(obj_path)
print('faces shape: ', mesh.faces.shape)
sample_num = 512
area, prob, uniform_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, sample_num)

# Visualization. For you to check your code
np.savetxt('uniform_sampling_vis.txt', uniform_pc)

print('area shape: ',area.shape)
print('prob shape: ',prob.shape)
print('pc shape: ',uniform_pc.shape)
# the result should satisfy: 
#       area.shape = (13712, ) 
#       prob.shape = (13712, ) 
#       uniform_pc.shape = (512, 3) 

# For submission
save_dict = {'area': area, 'prob': prob, 'pc': uniform_pc}
np.save('../results/uniform_sampling_results', save_dict)

faces shape:  (13712, 3)
area shape:  (13712,)
prob shape:  (13712,)
pc shape:  (512, 3)


In [5]:
# task 2: FPS

init_sample_num = 2000
final_sample_num = 512
_,_, tmp_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, init_sample_num)
fps_pc = farthest_point_sampling(tmp_pc, final_sample_num)

# Visualization. For you to check your code
np.savetxt('fps_vis.txt', fps_pc)

# For submission
np.save('../results/fps_results', fps_pc)

In [18]:
# task 3: metrics
import sys
from earthmover.earthmover import earthmover_distance   # EMD may be very slow (1~2mins)
# -----------TODO---------------
# compute chamfer distance and EMD for two point clouds sampled by uniform sampling and FPS.
# sample and compute CD and EMD again. repeat for five times.
# save the mean and var.
def c_d(pc_1, pc_2):
    def one_way_chamfer_distance(pc_1, pc_2):
        distance = np.sum((pc_1[:, np.newaxis, :] - pc_2) ** 2, axis = 2) ** 0.5
        one_way_CD = np.sum(distance.min(axis = 1))
        return one_way_CD
    CD = one_way_chamfer_distance(pc_1, pc_2) + one_way_chamfer_distance(pc_2, pc_1)
    return CD

def em_d(pc_1, pc_2):
    sto = sys.stdout
    fd = open('/dev/null', 'w')
    sys.stdout = fd
    EMD = earthmover_distance(pc_1, pc_2)
    sys.stdout = sto
    fd.close()
    return EMD

CD, EMD = [], []
for _ in range(5):
    _, _, pc_1 = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, final_sample_num)
    _, _, pc_2_tmp = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, init_sample_num)
    pc_2 = farthest_point_sampling(pc_2_tmp, final_sample_num)
    CD.append(c_d(pc_1, pc_2))
    EMD.append(em_d([tuple(pc) for pc in pc_1.tolist()], [tuple(pc) for pc in pc_2.tolist()]))

CD_mean = np.mean(CD)
CD_var = np.var(CD)
EMD_mean = np.mean(EMD)
EMD_var = np.var(EMD)
# -----------TODO---------------

# For submission
np.save('../results/metrics', {'CD_mean':CD_mean, 'CD_var':CD_var, 'EMD_mean':EMD_mean, 'EMD_var':EMD_var})